# Libraries

In [7]:
!pip install beautifulsoup4
!pip install pandas
!pip install --upgrade google-auth google-auth-oauthlib google-api-python-client
# Clear 
!python -c "import google; print('Google library is installed')"



Google library is installed


In [10]:
!python -m venv__proj_job_agg .venv


C:\Users\MalikW\AppData\Local\Programs\Python\Python313\python.exe: No module named venv__proj_job_agg


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Google library is installed


# Google Authentication

In [5]:
import os
import pickle
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

# Define the scope for Gmail API
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def authenticate_gmail():
    """Authenticate and get the Gmail API service"""
    creds = None

    # Check if token.pickle already exists for storing user credentials
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # If no valid credentials, ask user to log in
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES
            )
            # Explicitly set redirect_uri for debugging
            flow.redirect_uri = "http://localhost:8080/"
            creds = flow.run_local_server(port=8080)
        # Save credentials for next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    # Build the Gmail API service
    return build('gmail', 'v1', credentials=creds)


if __name__ == '__main__':
    try:
        # Authenticate and initialize the Gmail API
        service = authenticate_gmail()
        print("Gmail API authenticated successfully!")
    except Exception as e:
        print("Error during authentication:", e)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=416298047261-pgh55f954cr71omfj22djkcudkjh8t36.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=IAkRSEcC7Qy40VTmC9uOa1doNL7htB&access_type=offline
Gmail API authenticated successfully!


# Sample email saved as html webpage 
So we can judge the struture and write the logic

In [6]:
import base64

def fetch_email_and_save_as_html(service):
    """Fetch the top email from <alert@indeed.com> and save it as an HTML file."""
    try:
        # Search query to find emails from <alert@indeed.com>
        query = "from:alert@indeed.com"
        results = service.users().messages().list(userId='me', q=query, maxResults=1).execute()

        messages = results.get('messages', [])

        if not messages:
            print("No emails found from <alert@indeed.com>.")
            return

        # Fetch the top email
        message = service.users().messages().get(userId='me', id=messages[0]['id']).execute()

        # Extract email body
        payload = message['payload']
        parts = payload.get('parts', [])

        email_body = None

        # Iterate through parts to find text/html
        for part in parts:
            mime_type = part.get('mimeType', '')
            if mime_type == 'text/html' and 'body' in part and 'data' in part['body']:
                email_body = decode_email_body(part['body']['data'])
                break

        if not email_body:
            print("No HTML body found in the email.")
            return

        # Save the email body as an HTML file
        filename = "indeed_alert.html"
        with open(filename, "w", encoding="utf-8") as file:
            file.write(email_body)
        print(f"Email saved as {filename}. Open this file in a browser to examine it.")

    except Exception as e:
        print("An error occurred while fetching the email:", e)


def decode_email_body(encoded_body):
    """Decode base64 encoded email body."""
    try:
        decoded_bytes = base64.urlsafe_b64decode(encoded_body)
        return decoded_bytes.decode('utf-8')
    except Exception as e:
        print("Error decoding email body:", e)
        return "Error decoding body."


# Assuming `service` is already authenticated and available
if __name__ == '__main__':
    # Fetch the email and save it as an HTML file
    fetch_email_and_save_as_html(service)


Email saved as indeed_alert.html. Open this file in a browser to examine it.


# Sample Fetching the data as needed 

In [7]:
from bs4 import BeautifulSoup

def extract_job_details(html_file):
    """Extract job details like search term, company name, job title, location, and posting date."""
    with open(html_file, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

    # Debugging: Print all <p> tags
    print("Debugging <p> Tags:")
    for p_tag in soup.find_all('p'):
        print("Raw HTML:", p_tag)
        print("Text Content:", p_tag.get_text())
        print("\n")

    # Extract Search Term
    search_term = extract_search_term(soup)

    jobs = []

    # Extract job listings
    job_listings = soup.find_all('a', style=lambda t: t and 'font-size:16px' in t)

    for job in job_listings:
        job_details = {}

        # Job Title
        job_details['Job Title'] = job.text.strip()

        # Company Name
        company_section = job.find_next('span', style=lambda t: t and 'font-weight:700' in t)
        job_details['Company'] = company_section.text.strip() if company_section else 'Unknown'

        # Location
        location_section = company_section.find_next('span', style=lambda t: t and 'font-size:14px' in t) if company_section else None
        job_details['Location'] = location_section.text.strip() if location_section else 'Unknown'

        # Posting Date
        posting_date_section = job.find_next('td', style=lambda t: t and 'font-family' in t)
        job_details['Posted'] = posting_date_section.text.strip() if posting_date_section else 'Unknown'

        jobs.append(job_details)

    return search_term, jobs


def extract_search_term(soup):
    """Extract the search term from the email."""
    # Search for the paragraph containing the text directly
    alert_paragraph = soup.find(string=lambda text: 'Your job alert is active' in text if text else False)
    
    if alert_paragraph:
        # Check for sibling or child elements containing the search term
        parent = alert_paragraph.find_parent()
        if parent:
            bold_tags = parent.find_all('b')
            if bold_tags and len(bold_tags) >= 1:
                return bold_tags[0].get_text().strip()

    return "Search term not found"


if __name__ == '__main__':
    # Path to your Indeed email HTML file
    html_file = 'indeed_alert.html'
    search_term, jobs = extract_job_details(html_file)

    print(f"\nSearch Term: {search_term}\n")
    print("Jobs Found:")
    for job in jobs:
        print(job)


Debugging <p> Tags:
Raw HTML: <p align="center" class="r-12" style="font-family:'Noto Sans', Helvetica, Arial, sans-serif;font-size:14px;line-height:20px;color:#767676;Margin:0">Refined by: within 100 kilometres, CIBC</p>
Text Content: Refined by: within 100 kilometres, CIBC


Raw HTML: <p align="center" class="r-11" style="font-family:'Noto Sans', Helvetica, Arial, sans-serif;font-size:14px;line-height:20px;color:#767676;Margin:0">These job ads match your saved job alert ¹</p>
Text Content: These job ads match your saved job alert ¹


Raw HTML: <p align="center" style="font-size:14px;color:#2D2D2D;Margin:0">View jobs: <a href="https://ca.indeed.com/jobs?q=data+and+cibc&amp;hl=en&amp;from=ja&amp;l=Toronto%2C+ON&amp;radius=100&amp;alid=6760aa764a0da4183f947ebb&amp;sc=0kf%3Afcckey%282f81ab34c520110f%29%3B&amp;tmtk=1ifc3k1fog48n800&amp;utm_campaign=job_alerts&amp;utm_medium=email&amp;utm_source=jobseeker_emails&amp;of=1&amp;fr=b&amp;fromage=1" style="color:#2557A7;font-size:14px;font-weig

In [8]:
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime, timedelta

def extract_job_details(html_file):
    """Extract and clean job details."""
    with open(html_file, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

    # Extract Search Term
    search_term = extract_search_term(soup)

    # Extract Refined By
    refined_by = extract_refined_by(soup)

    jobs = []
    script_execution_date = datetime.now().strftime("%Y-%m-%d")

    # Extract Job Listings
    job_listings = soup.find_all('a', href=True, style=lambda x: x and 'font-size:16px' in x)

    for job in job_listings:
        job_details = {}

        # Job Title
        job_title = job.text.strip()
        if not job_title or "View all jobs" in job_title or "Edit" in job_title:
            continue  # Skip rows with unwanted titles

        # Company Name
        company_section = job.find_next('span', class_='r-g')
        company = company_section.text.strip() if company_section else "Unknown"
        company = re.sub(r'\s*\d+\.\d+\s*', '', company)  # Remove ratings like "3.9"

        # Location
        location_section = job.find_next('span', class_='r-h')
        location = location_section.text.replace('-', '').strip() if location_section else "Unknown"

        # Posted
        posted_section = job.find_next('td', class_='r-k')
        posted = posted_section.text.strip() if posted_section else "Unknown"

        # Calculate Date of Post
        date_of_post = calculate_date_of_post(posted)
        day_of_week = calculate_day_of_week(date_of_post)

        # Add Data
        job_details['Search Term'] = search_term
        job_details['Refined By'] = refined_by
        job_details['Job Title'] = job_title
        job_details['Company'] = company
        job_details['Location'] = location
        job_details['Days Since Posted'] = posted
        job_details['Date of Post'] = date_of_post
        job_details['Day'] = day_of_week
        job_details['Script Execution Date'] = script_execution_date

        jobs.append(job_details)

    return jobs


def extract_search_term(soup):
    """Extract search term dynamically from the entire HTML text."""
    try:
        full_text = soup.get_text(separator=" ", strip=True)
        match = re.search(r"for\s+(.*?)\s+in\s+", full_text)
        if match:
            return match.group(1).strip()
        return "Search term not found"
    except Exception as e:
        print("Error extracting search term:", e)
        return "Unknown"


def extract_refined_by(soup):
    """Extract 'Refined By' details."""
    try:
        refined_section = soup.find('p', string=lambda text: text and "Refined by" in text)
        if refined_section:
            return refined_section.text.replace("Refined by: ", "").strip()
        return "Refined by not found"
    except Exception as e:
        print("Error extracting refined by:", e)
        return "Unknown"


def calculate_date_of_post(posted_text):
    """Calculate the exact date based on the 'Posted' text."""
    today = datetime.now()
    if "day" in posted_text:
        days_ago = int(re.search(r'\d+', posted_text).group())
        return (today - timedelta(days=days_ago)).strftime("%Y-%m-%d")
    elif "hour" in posted_text:
        hours_ago = int(re.search(r'\d+', posted_text).group())
        return (today - timedelta(hours=hours_ago)).strftime("%Y-%m-%d")
    elif "week" in posted_text:
        weeks_ago = int(re.search(r'\d+', posted_text).group())
        return (today - timedelta(weeks=weeks_ago)).strftime("%Y-%m-%d")
    else:
        return "Unknown"


def calculate_day_of_week(date_string):
    """Calculate the day of the week based on a date string."""
    try:
        date_obj = datetime.strptime(date_string, "%Y-%m-%d")
        return date_obj.strftime("%A")
    except Exception:
        return "Unknown"


if __name__ == '__main__':
    html_file = 'indeed_alert.html'

    # Extract job details
    jobs = extract_job_details(html_file)

    # Convert to DataFrame
    df = pd.DataFrame(jobs)

    print("\nCleaned Job Details:")
    print(df)

    # Save to CSV
    output_file = "cleaned_job_alerts_with_dates_and_days.csv"
    df.to_csv(output_file, index=False)
    print(f"\nJob details saved to {output_file}")



Cleaned Job Details:
Empty DataFrame
Columns: []
Index: []

Job details saved to cleaned_job_alerts_with_dates_and_days.csv


# **Explanation of Parsing Logic**

The script uses **BeautifulSoup** to parse and analyze the structure of an HTML email. Parsing involves reading through the HTML, identifying key components, and extracting the desired information.

---

## **1. Reading the HTML File**
- The script reads the raw HTML content of the Indeed email using **BeautifulSoup**.  
- **HTML content** is broken into a tree-like structure of nested tags (e.g., `<p>`, `<span>`, `<a>`, etc.) that represent elements like job titles, company names, and search filters.

---

## **2. Finding Specific Tags and Elements**

To extract data, the script uses **tag searching** and **selectors**:

### **a. Job Listings**  
- **Target Element**: Job titles are located inside `<a>` tags with specific inline styles (`font-size:16px`).
- **Search Method**:  
   The script uses BeautifulSoup to search for all `<a>` tags where the style matches the expected pattern.  
   This isolates the clickable links that represent job titles.

### **b. Company Name**  
- **Target Element**: Company names are usually stored in `<span>` tags with specific classes (e.g., `r-g`).
- **Search Method**:  
   Once a job title is found, the script searches for the **next sibling `<span>` tag**.  
   This is based on the assumption that company names appear immediately after the job title in the HTML structure.

### **c. Location**  
- **Target Element**: Locations are often stored in `<span>` tags or inline elements immediately following the company name.  
- **Search Method**:  
   The script looks for the **next sibling `<span>`** after the company name.  
   If a dash (`-`) is present, the script cleans the text to isolate the location (e.g., "Toronto, ON").

### **d. Days Since Posted**  
- **Target Element**: Posting dates (e.g., "2 days ago") are located in `<td>` tags with specific classes like `r-k`.
- **Search Method**:  
   After finding the job title and company, the script navigates to the **next `<td>`** element to extract this information.

---

## **3. Extracting Search Term**
The **search term** is extracted dynamically by scanning the HTML content for specific text patterns:

- **Target Sentence**: `"You'll receive your first daily job alert for ... in Toronto, ON."`
- **Method**:
   - The script searches through the entire HTML as plain text.
   - Using **regex**, it identifies and extracts the portion of the sentence between `"for"` and `"in"`.  
     For example:
     - **Sentence**: "You'll receive your first daily job alert for data and rbc in Toronto, ON."
     - **Extracted Term**: `data and rbc`

---

## **4. Parsing Relative Dates**
Relative posting times like **"2 days ago"** or **"3 weeks ago"** are dynamically parsed to compute exact dates.

- **Method**:
   - The script identifies the relative time using patterns like:
      - `\d+ day(s) ago` → Extracts "2" from "2 days ago".
   - Subtracts the corresponding number of days or weeks from today’s date using Python’s `datetime` library.

For example:
- **Input**: "2 days ago"
- **Today's Date**: 2024-06-16
- **Result**: 2024-06-14

---

## **5. Cleaning and Processing Text**
HTML data often contains unwanted content like:
- Extra spaces
- Ratings (e.g., "3.9") included with company names
- Unnecessary symbols like dashes (`-`)

**Method**:
- The script uses **regular expressions (regex)** and string methods to clean the text:
   - Remove ratings: `Royal Bank of Canada 3.9` → `Royal Bank of Canada`
   - Remove extra dashes: `" - Toronto, ON"` → `"Toronto, ON"`
   - Strip extra spaces and line breaks.

---

## **6. Navigating Between Tags**
HTML elements are often nested or sequentially structured. The script uses **navigation techniques** to move between tags:

- **Next Sibling**:  
   For example, after finding a job title, the script uses `find_next()` to locate the **company name** and then the **location**.
- **Tag Attributes**:  
   Tags like `<a>` or `<span>` often have unique attributes (e.g., `style` or `class`) that the script uses to precisely identify the correct elements.
- **Text Content**:  
   Once the correct tag is located, the script extracts only the textual content inside it (ignoring other nested HTML tags).

---

## **7. Summary of Parsing Logic**
The script effectively breaks the HTML into its components, navigates through the structure, and extracts the required information:

1. **Job Titles** → `<a>` tags with styles.
2. **Company Names** → Next `<span>` tag with a specific class.
3. **Location** → Next `<span>` tag after the company name.
4. **Posted Dates** → `<td>` elements with specific classes.
5. **Search Term** → Extracted using regex from the entire text content.
6. **Text Cleaning** → Regular expressions remove noise like ratings and unnecessary characters.

---

## **Why It Works**
- The script uses a combination of **structured navigation** (moving between tags) and **text parsing** (regex) to extract data.
- It is **robust**: If one tag fails, the script falls back on defaults like `"Unknown"`.
- It works dynamically with both relative dates ("2 days ago") and plain text patterns.

This approach ensures clean, structured, and enriched data is extracted from even messy or inconsistent HTML.

Let me know if you need further clarifications on any specific part! 🚀


# --------------------------------------------------------------

# On email recive trigger the parser - Active Listner
This listens to all the new eamil from a spcified sender

In [9]:
import time
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Define the sender and query for filtering
SEARCH_SENDER = "malikhq27@gmail.com"

def list_new_emails(service):
    """List new emails from a specific sender and print their titles."""
    try:
        # Search for emails matching the query (e.g., from:alert@indeed.com)
        query = f"from:{SEARCH_SENDER} is:unread"
        results = service.users().messages().list(userId='me', q=query).execute()
        messages = results.get('messages', [])

        if not messages:
            print("No new emails found.")
            return

        print(f"Found {len(messages)} new email(s) from {SEARCH_SENDER}.")
        for message in messages:
            # Fetch email details
            msg = service.users().messages().get(userId='me', id=message['id']).execute()
            headers = msg['payload']['headers']

            # Extract subject from email headers
            subject = next((header['value'] for header in headers if header['name'] == 'Subject'), "No Subject")
            print(f"New Email Title: {subject}")

            # Mark the email as read (optional)
            service.users().messages().modify(
                userId='me',
                id=message['id'],
                body={"removeLabelIds": ["UNREAD"]}
            ).execute()

    except HttpError as error:
        print(f"An error occurred: {error}")


if __name__ == '__main__':
    try:
        # Authenticate using Gmail API
        service = authenticate_gmail()
        print("Gmail API authenticated successfully!")

        # Continuously check for new emails
        while True:
            list_new_emails(service)
            print("Waiting for new emails...")
            time.sleep(10)  # Poll every 10 seconds

    except Exception as e:
        print("Error:", e)


Gmail API authenticated successfully!
No new emails found.
Waiting for new emails...
No new emails found.
Waiting for new emails...
No new emails found.
Waiting for new emails...
No new emails found.
Waiting for new emails...


KeyboardInterrupt: 